## Quality Complaints Classification

ML model to classify whether a quality complaint is valid or not.

In [4]:
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split

from helpers import constants

pd.set_option('display.max_columns', None)

### EDA

In [5]:
df = pd.read_csv('db/dataset-qc.csv', sep=';', usecols=constants.COLUMNS)

In [6]:
df.head()

,brand,hybrid,batch,sieve,treatment,reason,result,quantity,city,state,germination
0,0,0,0,0,0,Insect Attack,0,20.0,0,MG,98.0
1,0,1,1,1,1,Insect Attack,0,3.0,1,MS,99.0
2,0,1,2,1,2,Insect Attack,0,27.0,1,MS,99.0
3,0,2,3,2,2,Insect Attack,0,382.0,2,MT,99.0
4,0,3,4,3,2,Insect Attack,0,100.0,2,MT,99.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2584 entries, 0 to 2583
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   brand        2584 non-null   int64  
 1   hybrid       2584 non-null   int64  
 2   batch        2584 non-null   int64  
 3   sieve        2584 non-null   int64  
 4   treatment    2584 non-null   int64  
 5   reason       2584 non-null   object 
 6   result       2584 non-null   int64  
 7   quantity     2547 non-null   float64
 8   city         2584 non-null   int64  
 9   state        2515 non-null   object 
 10  germination  2097 non-null   float64
dtypes: float64(2), int64(7), object(2)
memory usage: 222.2+ KB
